# NPH Absolute Quant Sample Info

This notebook populates, calculates, and concatenates NPH information starting from a .csv file obtained from the Core that contains NPH sample info. The output is a .txt file that will be uploaded into the appropriate Qiita Study sample information metadata file.

>--------------------------------------------------
> ### Before starting:
> 
> #### * GET LATEST LOT INFORMATION 
>           - Mackenzie will have this!
> #### * MAKE A NEW COPY OF THIS NOTEBOOK FOR EACH SAMPLE SHIPMENT
>--------------------------------------------------

## Inputs

### Enter the path to the .csv file from the Core:

In [ ]:
core_metadata_fp = None

### Enter the path where the output file should be saved :

In [ ]:
output_fp = None

### Enter the lot number for the current lot of storage liquid:

In [ ]:
storage_liquid_lot_number = None

#### IF the storage liquid has changed, update the following information: 

In [ ]:
storage_liquid_type = 'DNA/RNA Shield'

In [ ]:
# Never change the below for NPH, because stool is always the sample type
sample_density_g_ml_key = "density_stool_standardized_g_ml"

## Calculations

In [ ]:
import pandas as pd
import metapool.abs_quant_sample_info

In [ ]:
# Step 1: Import .csv file from Core and check input is as expected:

In [ ]:
# Import and define df
calc_df = pd.read_csv(core_metadata_fp)

In [ ]:
# Check that calc_df looks as expected
calc_df.head()

In [ ]:
# List column names that are currently present in calc_df
list(calc_df.columns)

In [ ]:
# Check for tube_id and/or sample_name columns are present in file (this might change depending on what col name we move forward with)
if 'tube_id' in calc_df.columns or 'sample_name' in calc_df.columns:
   print("tube_id and/or sample_name column is present in file : OK to proceed")
else:
   print("tube_id and/or sample_name column NOT present in file : DO NOT to proceed")

In [ ]:
# Step 2: Generate calculated values and add new columns to calc_df:

In [ ]:
calc_df = metapool.abs_quant_sample_info.add_abs_quant_metadata(
    calc_df, sample_density_g_ml_key, storage_liquid_type,
    storage_liquid_lot_number)

In [ ]:
# Step 3: Check output is as expected and export .csv:

In [ ]:
# Check new columns are present and expected
calc_df.head()

In [ ]:
# Export .txt
calc_df.to_csv(output_fp, sep='\t', index=False)